This example is very similar to the Simple Gradient Sweep, with two additoins:
- The animation alternates directions, and
- The animation gradient-sweeps between primary colors on the color wheel

In [ ]:
import dataclasses
import itertools
import random
import sys

import adafruit_dotstar
import board


from pixelbuf_pi_animation.data import *
from pixelbuf_pi_animation.player import SimplePixelBufPlayer

In [ ]:
# Constants
num_leds = 11
color_list = [
    (0, 255, 0),
    (0, 0, 255),
    (180, 0, 255),
    (255, 0, 0),
    (255, 40, 0),
    (255, 150, 0),
]

pixels = adafruit_dotstar.DotStar(board.SCLK, board.MOSI, num_leds, pixel_order=adafruit_dotstar.GBR, auto_write=False)

In [ ]:
color_cycle = itertools.cycle(color_list)
frames = []
direction_cycle = True

ending_color = color_cycle.__next__()
for t_ix in range(len(color_list)):
    gradient_pixels = []
    
    starting_color = ending_color
    ending_color = color_cycle.__next__()
    direction_cycle = not direction_cycle
    
    color_deltas = (ending_color[0]-starting_color[0], 
                ending_color[1]-starting_color[1], 
                ending_color[2]-starting_color[2])

    color_steps = (color_deltas[0] / num_leds,
                   color_deltas[1] / num_leds,
                   color_deltas[2] / num_leds)
    
    for i in range(num_leds):
        gradient_pixels.append(Pixel(
            red=min(starting_color[0] + (color_steps[0] * i), 255),
            green=min(starting_color[1] + (color_steps[1] * i), 255),
            blue=min(starting_color[2] + (color_steps[2] * i), 255),
            brightness=1.0
        ))

    
    for i in range(num_leds + 1):
        frame_pixels = [None] * num_leds
        for j in range(i):
            if direction_cycle:
                frame_pixels[j] = gradient_pixels[j]
            else:
                frame_pixels[num_leds-j-1] = gradient_pixels[j]
        frames.append(Frame(pixels=frame_pixels, display_ms=100))

animation = Animation(frames=frames, max_plays=1,loop_infinitely=False, pause_between_play_ms=0)
player = SimplePixelBufPlayer(controller=pixels)
player.load(animation)
player.play()